In [1]:
import pandas as pd
%matplotlib inline
import seaborn as sns
import numpy as np

In [2]:
units = pd.read_csv("../data/510049986/raw_data.csv", encoding="windows-1252", index_col=0)

C:\Users\Alex\Anaconda3\envs\wargame-analysis\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7,19,32,40,56,57,65,72,73,89,91,95,97,98,102,106,108,109,111,112,113,114,115,116,119,120,121,122,126,127,130,136,138,139,140,141,142,143,147,151,153,154,156,157,161,162,163,164,165,166,168,169,173,177,179,180,182,183,184,185,186,187,190,191,192,193,197,198,201,207,209,210,211,212,213,214,218,222,225,227,232,233,234,235,236,237,239,240,244,248,250,251,253,254,255,256,257,258,261,262,263,264,268,269,272,278,280,281,282,283,284,285,289,293,296,298,303,304,305,306,307,308,310,311,315,319,321,322,324,325,326,327,328,332,333,334,335,339,340,343,351,352,353,354,355,356,360,364,366,367,369,370,374,375,376,377,378,379,381,382,386,390,392,393,395,396,397,398,399,403,404,405,406,410,411,414,422,423,424,425,426,427,431,435,437,438,440,441,445,446,447,448,449,450,460,474,475,491,516,517,519,535,537,549,550,591,606,607,610,612,615,624,625,662,667,669,681,685,687,697,698,731,7

In [3]:
units_ = units[~units.apply(lambda srs: 'Deprec!' in srs['NameInMenu'], axis='columns')]

In [4]:
units_[['NameInMenu', 'Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2PhysicalDamages', 'Weapon2Arme']]

,NameInMenu,Weapon1PhysicalDamages,Weapon1Arme,Weapon2PhysicalDamages,Weapon2Arme
0,DANA,7,3,null,3
1,RZP KUB-M,9,3,NaN,NaN
2,2K22 TUNGUSKA-M,1,3,7,3
3,2K22 TUNGUSKA,1,3,7,3
4,2S15 NOROV,1,17,3,3
5,2S19 MSTA-S,7,3,null,3
6,2S1 GO?DZIK,6,3,null,3
7,2S1 GVOZDIKA,6,3,null,3
8,2S23 NONA-SVK,5,3,null,3
9,2S3M AKATSIYA,7,3,null,3


In [5]:
import re

def weaponify(unit):
    # We're going to build a new pandas Series with all of the information we want.
    srs = pd.Series()
    
    # First thing first, how many weapons are we talking about? A quick regex will get us this fact.
    try:
        top_weapon_number = max([int(re.search(r'\d+', c).group()) for c in unit.index\
                                 if ("Weapon" in c) and (pd.notnull(unit[c]))])
    except ValueError:
        # There are none!
        top_weapon_number = 0
    
    # Some things we need to keep track of outside of the loop.
    salvo_stock_indices_already_visited = []
    
    for i in range(1, top_weapon_number + 1):
#         import pdb; pdb.set_trace()
        
        fragment = 'Weapon{0}'.format(i)
        salvo_index = 0 if unit[fragment + 'SalvoStockIndex'] == "null" else int(unit[fragment + 'SalvoStockIndex'])
        
        # If we haven't already visited this stock index, this weapon is the first one there.
        # Let's copy its data into our new Series!
        if salvo_index not in salvo_stock_indices_already_visited:
            columns_of_interest = [c for c in unit.index if fragment in c]
            for c in columns_of_interest:
                srs[c] = unit[c]
        # If we have already visited this stock index, let's find out why.
        else:
            # Before we begin, let's find that previous weapon.
            # We can find it by taking the index of the previous weapon's position in the visited list and adding one.
            previous_fragment = "Weapon{0}".format(salvo_stock_indices_already_visited.index(salvo_index) + 1)
            
            # Case 1: The new (current) weapon is a smoke weapon. 
            # We see this when Arme == 3 and PhysicalDamages is "null".
            if (unit[fragment + 'Arme'] == "null" or int(unit[fragment + 'Arme']) == 3) \
                and (unit[fragment + 'PhysicalDamages'] == "null"):
                # Attach a can-smoke tag to the weapon.
                srs[previous_fragment + 'CanSmoke'] = True
            # Case 2a: The first weapon is an SMG, and the second weapon is also an SMG.
            # SMGs get doubled accuracy when the fight is in CQC. It turns out that this is implemented within the
            # engine by, no joke, simply attaching another copy of the weapon to the unit which points to the same
            # exact thing except that accuracy is doubled.
            #
            # For an example, refer to the French Groupe Commandat command infantry unit, which is the 15th unit
            # in the list of them on the Modding Suite (at the moment at least).
            #
            # To detect this case, we check to see if both weapons are SMGs.
            #
            # Case 2b: Similarly, if the the weapon is a battle rifle, expect half accuracy in CQC.
            # Case: 2c: If the weapon is an assault rifle, expect the same accuracy. But the weapon gets split anyway!
            elif (unit[fragment + 'TypeArme'] == "D1D5010000000000" \
                    and unit[previous_fragment + 'TypeArme'] == "D1D5010000000000") or \
                (unit[fragment + 'TypeArme'] == "6ABCBA1C00000000" \
                    and unit[previous_fragment + 'TypeArme'] == "6ABCBA1C00000000") or \
                (unit[fragment + 'TypeArme'] == "5C99C7BA89E30B00" \
                    and unit[previous_fragment + 'TypeArme'] == "5C99C7BA89E30B00"):
                    # In this case we do...nothing! The data is already there.
                    pass
            # Case 3: The new (current) weapon is the HE or AP component of the previous weapon.
            # These can occur in either order, so we have to make sense of it by looking at what the values are.
            # Taking a maximum does the job nicely, but first we have to fix nullity.
            else:
                # Then, we play the game.
                prior_arme = srs[previous_fragment + 'Arme']
                if prior_arme == 'null': prior_arme = np.nan
                curr_arme = unit[fragment + 'Arme']
                if curr_arme == 'null': curr_arme = np.nan
                prior_physical = unit[previous_fragment + 'PhysicalDamages']
                if prior_physical == 'null': prior_physical = np.nan
                curr_physical = unit[fragment + 'PhysicalDamages']
                if curr_physical == 'null': curr_physical = np.nan
                # The rathered tortured semantics are necessary to work around numpy bitching about types.
                arme = int(np.nanmax(np.array([prior_arme, curr_arme]).astype(float)))
                physical = int(np.nanmax(np.array([prior_physical, curr_physical]).astype(float)))
                # Attach the new values.
                srs[previous_fragment + 'Arme'] = arme
                srs[previous_fragment + 'PhysicalDamages'] = physical

        # Finally, don't forget to add the stock index to the visited list!
        salvo_stock_indices_already_visited.append(salvo_index)

    # We may have produced empties along the way, populating fields in a pattern like ['Weapon1', 'Weapon3'] instead of
    # e.g. ['Weapon1', 'Weapon2']. This is much easier to fix afterwards, though, then during. So let's fix that now!
    # Start with the same regex as before...
    try:
        top_weapon_number = max([int(re.search(r'\d+', c).group()) for c in srs.index\
                                 if ("Weapon" in c) and (pd.notnull(srs[c]))])
    except:
        return srs
    
    # Assuming we have weapons at all (the duck test above caught the case that we don't), we keep going.
    attached_weapon_numbers = [i for i in range(1, top_weapon_number + 1) if 'Weapon{0}Arme'.format(i) in srs.index]
    
    # Figure out how to map the attached weapon numbers.
    trips = zip(attached_weapon_numbers, range(1, len(attached_weapon_numbers) + 1))
    
    # Map down.
    for origin, destination in trips:
        if origin == destination:
            pass
        else:
            srs.index = [c if (('Weapon' not in c) or (int(re.search(r'\d+', c).group()) != origin))\
                            else c.replace(re.search(r'\d+', c).group(), str(destination)) for c in srs.index]
    
    return srs

In [6]:
units.query('NameInMenu == "RECCE #reco2"').iloc[0][['Weapon3SalvoStockIndex']]

Weapon3SalvoStockIndex    1
Name: 207, dtype: object

In [7]:
weaponify(units.query('NameInMenu == "KUB-M"').iloc[0])[['Weapon1Arme', 'Weapon1PhysicalDamages',
                                                          'Weapon2Arme', 'Weapon2PhysicalDamages', 
                                                          'Weapon3Arme', 'Weapon3PhysicalDamages']]  # Single weapon test

Weapon1Arme                 3
Weapon1PhysicalDamages      9
Weapon2Arme               NaN
Weapon2PhysicalDamages    NaN
Weapon3Arme               NaN
Weapon3PhysicalDamages    NaN
dtype: object

In [8]:
weaponify(units.query('NameInMenu == "AMX-30"').iloc[0])[['Weapon1Arme', 'Weapon1PhysicalDamages',
                                                          'Weapon2Arme', 'Weapon2PhysicalDamages', 
                                                          'Weapon3Arme', 'Weapon3PhysicalDamages']]  # Single AP/HE test

Weapon1Arme                 43
Weapon1PhysicalDamages       3
Weapon2Arme               null
Weapon2PhysicalDamages     0.5
Weapon3Arme                NaN
Weapon3PhysicalDamages     NaN
dtype: object

In [9]:
weaponify(units.query('NameInMenu == "AMX-30B"').iloc[0])[['Weapon1Arme', 'Weapon1PhysicalDamages',
                                                          'Weapon2Arme', 'Weapon2PhysicalDamages', 
                                                          'Weapon3Arme', 'Weapon3PhysicalDamages']]  # Double HE-AP test

Weapon1Arme                 43
Weapon1PhysicalDamages       3
Weapon2Arme                  6
Weapon2PhysicalDamages       1
Weapon3Arme               null
Weapon3PhysicalDamages     0.5
dtype: object

In [10]:
weaponify(units.query('NameInMenu == "DANA"').iloc[0])[['Weapon1Arme', 'Weapon1PhysicalDamages',
                                                          'Weapon2Arme', 'Weapon2PhysicalDamages', 
                                                          'Weapon3Arme', 'Weapon3PhysicalDamages']]  # Smoke test

Weapon1Arme                  3
Weapon1PhysicalDamages       7
Weapon2Arme                 41
Weapon2PhysicalDamages       1
Weapon3Arme                  1
Weapon3PhysicalDamages    0.75
dtype: object

In [11]:
weaponify(units_.ix[18])[['Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2Arme', 
                          'Weapon3PhysicalDamages', 'Weapon3Arme']]  # Test: Assault Rifle

Weapon1PhysicalDamages       1
Weapon1Arme               null
Weapon2Arme                 58
Weapon3PhysicalDamages     0.5
Weapon3Arme               null
dtype: object

In [12]:
weaponify(units_.ix[17])[['Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2Arme', 
                          'Weapon3PhysicalDamages', 'Weapon3Arme']]  # Test: Battle Rifle

Weapon1PhysicalDamages       1
Weapon1Arme               null
Weapon2Arme                 52
Weapon3PhysicalDamages     0.5
Weapon3Arme               null
dtype: object

In [13]:
weaponify(units.iloc[15])[['Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2Arme', 'Weapon2PhysicalDamages',
                          'Weapon3PhysicalDamages', 'Weapon3Arme']]  # Test: SMG

Weapon1PhysicalDamages       1
Weapon1Arme               null
Weapon2Arme               null
Weapon2PhysicalDamages     0.5
Weapon3PhysicalDamages     NaN
Weapon3Arme                NaN
dtype: object

In [14]:
weaponify(units.query('NameInMenu == "#command UDALOY II"').iloc[0])[
    ['Weapon1Arme', 'Weapon1PhysicalDamages', 'Weapon2Arme', 'Weapon2PhysicalDamages',
     'Weapon3Arme', 'Weapon3PhysicalDamages', 'Weapon4Arme', 'Weapon4PhysicalDamages',
     'Weapon5Arme', 'Weapon5PhysicalDamages', 'Weapon6Arme', 'Weapon6PhysicalDamages',
     'Weapon7Arme', 'Weapon7PhysicalDamages', 'Weapon8Arme', 'Weapon8PhysicalDamages',
     'Weapon9Arme', 'Weapon9PhysicalDamages', 'Weapon10Arme', 'Weapon10PhysicalDamages',
     'Weapon11Arme', 'Weapon11PhysicalDamages',]]
# Test: biggest ship

Weapon1Arme                  3
Weapon1PhysicalDamages       8
Weapon2Arme                 54
Weapon2PhysicalDamages       6
Weapon3Arme                 54
Weapon3PhysicalDamages       6
Weapon4Arme                  3
Weapon4PhysicalDamages       5
Weapon5Arme                  3
Weapon5PhysicalDamages       5
Weapon6Arme                  3
Weapon6PhysicalDamages       4
Weapon7Arme                  3
Weapon7PhysicalDamages       1
Weapon8Arme                  3
Weapon8PhysicalDamages       4
Weapon9Arme                  3
Weapon9PhysicalDamages       1
Weapon10Arme               NaN
Weapon10PhysicalDamages    NaN
Weapon11Arme               NaN
Weapon11PhysicalDamages    NaN
dtype: object

In [15]:
weaponify(units.query('NameInMenu == "#command NAJIN"').iloc[0])[
    ['Weapon1Arme', 'Weapon1PhysicalDamages', 'Weapon2Arme', 'Weapon2PhysicalDamages',
     'Weapon3Arme', 'Weapon3PhysicalDamages', 'Weapon4Arme', 'Weapon4PhysicalDamages',
     'Weapon5Arme', 'Weapon5PhysicalDamages', 'Weapon6Arme', 'Weapon6PhysicalDamages',
     'Weapon7Arme', 'Weapon7PhysicalDamages', 'Weapon8Arme', 'Weapon8PhysicalDamages',
     'Weapon9Arme', 'Weapon9PhysicalDamages', 'Weapon10Arme', 'Weapon10PhysicalDamages',
     'Weapon11Arme', 'Weapon11PhysicalDamages']]
# Test: big ship.

Weapon1Arme                 3
Weapon1PhysicalDamages      6
Weapon2Arme                 3
Weapon2PhysicalDamages      6
Weapon3Arme                54
Weapon3PhysicalDamages      8
Weapon4Arme                54
Weapon4PhysicalDamages      8
Weapon5Arme                 3
Weapon5PhysicalDamages      1
Weapon6Arme                 3
Weapon6PhysicalDamages      1
Weapon7Arme                 3
Weapon7PhysicalDamages      1
Weapon8Arme                 3
Weapon8PhysicalDamages      1
Weapon9Arme                 3
Weapon9PhysicalDamages      1
Weapon10Arme                3
Weapon10PhysicalDamages     1
Weapon11Arme                3
Weapon11PhysicalDamages     1
dtype: object

In [16]:
weaponify(units.query('NameInMenu == "AML 60/20 SERVAL"').iloc[0])[
    ['Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2Arme', 'Weapon2PhysicalDamages',
     'Weapon3PhysicalDamages', 'Weapon3Arme']]
# Test: complex

Weapon1PhysicalDamages     2
Weapon1Arme                3
Weapon2Arme               38
Weapon2PhysicalDamages     1
Weapon3PhysicalDamages     1
Weapon3Arme                6
dtype: object

In [17]:
weaponify(units.query('NameInMenu == "AH-64A APACHE"').iloc[0])[
    ['Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2Arme', 'Weapon2PhysicalDamages',
     'Weapon3PhysicalDamages', 'Weapon3Arme']]
# Test: complex

Weapon1PhysicalDamages     1
Weapon1Arme                6
Weapon2Arme                3
Weapon2PhysicalDamages     2
Weapon3PhysicalDamages     1
Weapon3Arme               60
dtype: object

In [18]:
from checkpoints import checkpoints
checkpoints.enable()

units__ = units_.safe_apply(weaponify, axis='columns')

In [19]:
units__.to_csv("../data/510049986/intermediate_data.csv")